# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
# Reading a test image file 
image = mpimg.imread('test_images/solidWhiteRight.jpg')
# Printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
# if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')
plt.imshow(image)  

## Ideas for Lane Detection Pipeline

**Some OpenCV functions

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images  
`cv2.cvtColor()` to grayscale or change color  
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more interesting functionalities**

## Helper Functions

In [ ]:
import math
from collections import deque

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
#     return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    # Defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    # Defining a 3-channel or 1-channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    # Filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    # Returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    
    # Uncomment the following line in case you want to see the masked result in grayscale  
    # plt.imshow(masked_image, cmap='gray')
    return masked_image

def linear_regression(lines_, remove_outliers=True):
    global Error_threshold  # This Error_threshold is later used to filter out outliers in lane detection
    length_ = len(lines_)
    
    if length_ < 2:
        return 0, 0
    
    x_bar = sum([item[0] for item in lines_]) / length_
    y_bar = sum([item[1] for item in lines_]) / length_
    # Slope
    m_ = sum([(item[1] - y_bar) * (item[0] - x_bar) for item in lines_]) / sum([(item[0] - x_bar)**2 for item in lines_])
    # y-intercept
    b_ = y_bar - m_ * x_bar

    if remove_outliers:
        # Finding errors
        errors_ = [abs(m_ * p[0] + b_ - p[1]) for p in lines_]
        
        # Finding outliers based on Error_threshold
        Q3_ = np.percentile(errors_ , Error_threshold)

        # Filtering out outliers from Left lines 
        lines_ = [lines_[i] for i in range(length_) if errors_[i] < Q3_]
        # Running regression without outliers
        return linear_regression(lines_, False)
    
    else: 
        return m_, b_


def draw_lines(img, lines, color=[255, 0, 0], thickness=8):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """    
    # Keeping midpoints of all lines in the right and left lanes in separate deques
    linesL = deque()
    linesR = deque()
    
    # Separating and Storing midpoints of L/R lines based on their location on the image
    for line in lines:
        for x1,y1,x2,y2 in line:
                if x1 < img_width // 2 and x2 < img_width // 2:
                    linesL.append(((x1 + x2) // 2, (y1 + y2) // 2))
                elif x1 > img_width // 2 and x2 > img_width // 2:
                    linesR.append(((x1 + x2) // 2, (y1 + y2) // 2))
    
    # Left lane linear regression
    # Slope, y-intercept, errors
    mL, bL = linear_regression(linesL)
    
    # Right lane linear regression
    # Slope, y-intercept, errors
    mR, bR = linear_regression(linesR)
       
    # yt and ys are upper and lower bounds of the mask region, they are used as upper/lower limits for line extrapolation
    yt = P2[1]
    ys = img_height
      
    # Drawing extrapolation of Left and Right lanes
    if not(math.isnan(mL)) and mL != 0:
        cv2.line(img, (int((ys - bL) / mL), ys), (int((yt - bL) / mL), yt), color, thickness)
    if not(math.isnan(mR)) and mR != 0:
        cv2.line(img, (int((ys - bR) / mR), ys), (int((yt - bR) / mR), yt), color, thickness)
    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img


def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
import time
images = ['test_images/' + img for img in os.listdir("test_images/")]

# Gaussian smoothing parameters
kernel_size = 7
# Canny parameters
low_threshold = 125
high_threshold = 140
# Defining Hough transform parameters, iterating over the output lines and drawing lines on a blank image
rho = 2
theta = np.pi/180
threshold = 2
min_line_len = 10
max_line_gap = 30


def process_image_test(image):
    global img_width, img_height, P2
    
    # [1] Converting image color space to grayscale
    gray = grayscale(image)
    
    # [2] Applying Gaussian smoothing
    blur_gray = gaussian_blur(gray, kernel_size)
    
    # [3] Applying Canny transform
    edges = canny(blur_gray, low_threshold, high_threshold)
    
    # [4] Defining a polygon region mask and use that as region of interest for lane detection
    img_width = image.shape[1]
    img_height = image.shape[0]
    
    poly_region = np.copy(image)*0
    P1 = (int(0.05 * img_width), img_height)
    P2 = (int(0.45 * img_width), int(0.624 * img_height))
    P3 = (int(0.56 * img_width), int(0.624 * img_height))
    P4 = (int(0.95 * img_width), img_height)
    P5 = (int(0.75 * img_width), P4[1])
    P6 = (int(0.52 * img_width), P3[1])
    P7 = (int(0.49 * img_width), P2[1])
    P8 = (int(0.25 * img_width), P1[1])

    # Uncomment the following lines in case you need to check the polygon region mask
    # cv2.line(poly_region,P1,P2,(0,255,0),5)
    # cv2.line(poly_region,P2,P3,(0,255,0),5)
    # cv2.line(poly_region,P3,P4,(0,255,0),5)
    # cv2.line(poly_region,P4,P5,(0,255,0),5)
    # cv2.line(poly_region,P5,P6,(0,255,0),5)
    # cv2.line(poly_region,P6,P7,(0,255,0),5)
    # cv2.line(poly_region,P7,P8,(0,255,0),5)
    # cv2.line(poly_region,P8,P1,(0,255,0),5)
    
    vertices = np.array([[P1, P2, P3, P4, P5, P6, P7, P8, P1]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
      
    # [5] Applying Hough transform, detect lanes and extend using extrapolation
    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)
    
    # [6] Drawing transparent lanes on the original image
    lines_edges = weighted_img(line_image, image, 0.8, 1., 0.)
    # In case you need to check the mask region
    lines_edges = weighted_img(poly_region, lines_edges, 0.8, 1., 0.) 

    return lines_edges
    
i = 1
img_width = None
img_height = None
P2 = None
Error_threshold = 70 # Allowable Error Percentage for filtering out outliers (defiend as percentile)
for img in images:
    # Loading image and converting to grayscale
    image = mpimg.imread(img)

    lines_edges = process_image_test(image)
    
    plt.figure(figsize=(80, 60)) 
    plt.subplot(6, 1, i)
    
    plt.imshow(lines_edges)
    plt.axis('off')
    plt.show()
    i += 1


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    global img_width, img_height, P2
    
    # [1] Converting image color space to grayscale
    gray = grayscale(image)
    
    # [2] Applying Gaussian smoothing
    blur_gray = gaussian_blur(gray, kernel_size)
    
    # [3] Applying Canny transform
    edges = canny(blur_gray, low_threshold, high_threshold)
    
    # [4] Defining a polygon region mask and use that as region of interest for lane detection
    img_width = image.shape[1]
    img_height = image.shape[0]
    
    poly_region = np.copy(image)*0
    P1 = (int(0.05 * img_width), img_height)
    P2 = (int(0.45 * img_width), int(0.624 * img_height))
    P3 = (int(0.56 * img_width), int(0.624 * img_height))
    P4 = (int(0.95 * img_width), img_height)
    P5 = (int(0.75 * img_width), P4[1])
    P6 = (int(0.52 * img_width), P3[1])
    P7 = (int(0.49 * img_width), P2[1])
    P8 = (int(0.25 * img_width), P1[1])

    # Uncomment the following lines in case you need to check the polygon region mask
    # cv2.line(poly_region,P1,P2,(0,255,0),5)
    # cv2.line(poly_region,P2,P3,(0,255,0),5)
    # cv2.line(poly_region,P3,P4,(0,255,0),5)
    # cv2.line(poly_region,P4,P5,(0,255,0),5)
    # cv2.line(poly_region,P5,P6,(0,255,0),5)
    # cv2.line(poly_region,P6,P7,(0,255,0),5)
    # cv2.line(poly_region,P7,P8,(0,255,0),5)
    # cv2.line(poly_region,P8,P1,(0,255,0),5)
    
    vertices = np.array([[P1, P2, P3, P4, P5, P6, P7, P8, P1]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
      
    # [5] Applying Hough transform, detect lanes and extend using extrapolation
    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)
    
    # [6] Drawing transparent lanes on the original image
    lines_edges = weighted_img(line_image, image, 0.8, 1., 0.)
    # In case you need to check the mask region
    lines_edges = weighted_img(poly_region, lines_edges, 0.8, 1., 0.) 

    return lines_edges

Let's try the one with the solid white lane on the right first ...

In [ ]:
Error_threshold = 70 # Allowable Error Percentage (defined as percentile)
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
Error_threshold = 70 # Allowable Error Percentage
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
Error_threshold = 70 # Allowable Error Percentage
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
# clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(5,6)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))